# Getting Start

### Installation of JanusQ
JanusQ can be installed by docker, wheel, and soruce code. Docker is recommended, as all functions have been tested on it.
Linux (Ubuntu 22.04 latest) platform and Python (3.10) is prefered when instaling by wheel or soruce code.

**From docker.**
Pull docker using docker 
        
        docker pull jxhhhh/janusq:latest

The code is in "/home/JanusQ-main". The examples that can be directly run is in "/home/JanusQ-main/examples"

**From wheel.**
Download janusq.whl from "JanusQ-main/dist".

        pip install janusq.whl

**From source code.**
Run following commends.

        git clone git@github.com:JanusQ/JanusQ.git
        cd JanusQ
        pip install -r requirements.txt

### Structure of JanusQ

- JanusCT
  - vectorization
  - fidelity preidiction
  - fidelity optimization
  - unitary decomposition
  - bug identification
- JanusFEM
  - readout calibration
- HyQSAT
  - solve sat domain problem
- time crystal

### Note
1. Permission denied 'minisat_core'  
chmod +x janusq/hyqsat/minisat_core

### Test JanusQ

In [1]:
import sys
sys.path.append('..')

#### Test Janus-CT

In [2]:
# test janus-ct
from janusq.data_objects.backend import LinearBackend
from janusq.data_objects.random_circuit import random_circuits
from janusq.analysis.vectorization import RandomwalkModel
# define a 10-qubit backend
n_qubits = 8
n_steps = 1
n_walks = 20
backend = LinearBackend(n_qubits, 1)

circuits = random_circuits(backend, n_circuits=300, n_gate_list=[30, 50, 100], two_qubit_prob_list=[.4], reverse=True)

vec_model = RandomwalkModel(n_steps = n_steps, n_walks = n_walks, backend = backend)
vec_model.train(circuits, multi_process=False, remove_redundancy = False)

2024-04-16 15:28:12,000 - INFO: start random walk for 300 circuits
100%|██████████| 300/300 [00:01<00:00, 150.39it/s]
2024-04-16 15:28:14,000 - INFO: count path
2024-04-16 15:28:14,030 - INFO: device size after random walk = 15
2024-04-16 15:28:14,031 - INFO: 0's path table size = 11
2024-04-16 15:28:14,032 - INFO: 1's path table size = 16
2024-04-16 15:28:14,033 - INFO: 2's path table size = 19
2024-04-16 15:28:14,034 - INFO: 3's path table size = 19
2024-04-16 15:28:14,035 - INFO: 4's path table size = 19
2024-04-16 15:28:14,036 - INFO: 5's path table size = 19
2024-04-16 15:28:14,036 - INFO: 6's path table size = 16
2024-04-16 15:28:14,037 - INFO: 7's path table size = 11
2024-04-16 15:28:14,038 - INFO: (0, 1)'s path table size = 13
2024-04-16 15:28:14,039 - INFO: (1, 2)'s path table size = 18
2024-04-16 15:28:14,040 - INFO: (3, 4)'s path table size = 21
2024-04-16 15:28:14,040 - INFO: (2, 3)'s path table size = 21
2024-04-16 15:28:14,041 - INFO: (6, 7)'s path table size = 13
2024-0

#### Test Janus-FEM

In [5]:
# test readout calibration

from janusq.simulator.noisy_simulator import NoisySimulator
from janusq.data_objects.backend import LinearBackend
from janusq.simulator.readout_error_model import ReadoutErrorModel
from janusq.optimizations.readout_mitigation.fem import  EnumeratedProtocol
from janusq.data_objects.algorithms import get_algorithm_circuits

n_qubits = 3
backend = LinearBackend(n_qubits, 1)
circuit = get_algorithm_circuits(n_qubits, backend, algs = ['ghz'])[0]

samples = 10000

simulator = NoisySimulator(
    backend, readout_error_model = ReadoutErrorModel.random_model(backend))
output_noise = simulator.execute(circuit,samples)


protocol = EnumeratedProtocol(n_qubits)
real_bstrs, circuits_protocol = protocol.gen_circuits()

all_statuscnts = [
    simulator.execute(cir_protocol, samples)
    for cir_protocol in circuits_protocol
]

print(all_statuscnts)


2024-04-16 15:31:05,325 - DEBUG: reading /root/.cache/python-entrypoints/76517c6b774723e905e862a702703d1bd6e41bacf4f4536448c2593ebaf3d912
2024-04-16 15:31:05,343 - DEBUG: writing to /root/.cache/python-entrypoints/76517c6b774723e905e862a702703d1bd6e41bacf4f4536448c2593ebaf3d912
2024-04-16 15:31:05,346 - DEBUG: found extension EntryPoint(name='default', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:DefaultInitPassManager', group='qiskit.transpiler.init')
2024-04-16 15:31:05,348 - DEBUG: found extension EntryPoint(name='default', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:DefaultLayoutPassManager', group='qiskit.transpiler.layout')
2024-04-16 15:31:05,349 - DEBUG: found extension EntryPoint(name='dense', value='qiskit.transpiler.preset_passmanagers.builtin_plugins:DenseLayoutPassManager', group='qiskit.transpiler.layout')
2024-04-16 15:31:05,350 - DEBUG: found extension EntryPoint(name='noise_adaptive', value='qiskit.transpiler.preset_passmanagers.builtin

[{'101': 70, '111': 2, '011': 54, '001': 772, '110': 43, '100': 856, '010': 446, '000': 7757}, {'110': 2, '100': 43, '010': 23, '000': 490, '111': 49, '011': 417, '101': 888, '001': 8088}, {'101': 90, '111': 8, '011': 44, '100': 806, '001': 850, '110': 49, '010': 470, '000': 7683}, {'101': 7, '100': 27, '001': 22, '111': 92, '011': 839, '110': 833, '000': 269, '010': 7911}, {'100': 1, '101': 24, '001': 281, '110': 51, '000': 13, '010': 460, '111': 866, '011': 8304}, {'101': 4, '100': 18, '001': 29, '110': 823, '111': 74, '011': 840, '000': 277, '010': 7935}, {'111': 3, '011': 33, '110': 51, '101': 99, '100': 836, '001': 791, '010': 443, '000': 7744}, {'110': 2, '100': 45, '111': 51, '011': 437, '101': 873, '010': 30, '000': 446, '001': 8116}, {'111': 5, '011': 50, '110': 50, '101': 84, '001': 789, '100': 849, '010': 411, '000': 7762}, {'001': 59, '011': 2, '111': 39, '101': 863, '110': 407, '100': 7976, '010': 33, '000': 621}, {'110': 30, '010': 2, '000': 35, '011': 36, '111': 493, '10

#### Test Janus-SAT

In [4]:
# test hyqsat calibration
# input cnf flie
file_path = "cnf_examples/uf50-01.cnf"
from janusq.hyqsat import readCNF

print(readCNF(file_path))

([(-3, 7, 36), (-3, -42, -48), (-41, -47, -49), (8, 17, -40), (-21, -31, -39), (-22, 36, 49), (14, 27, 38), (6, 15, -18), (6, 7, -43), (-7, 23, 34), (2, -13, 14), (2, -42, 47), (3, -33, -35), (40, 44, 49), (31, 36, 50), (-3, -36, -37), (26, -29, 43), (15, 29, -45), (-11, 18, 24), (6, -26, -47), (-10, -33, -50), (6, 16, 32), (-34, 37, 41), (7, -17, -28), (19, -44, 46), (7, 22, -48), (3, 34, 39), (31, -43, 46), (23, -27, 32), (-18, 37, -50), (5, 11, 20), (6, -24, -45), (-14, -23, -34), (20, 21, -22), (-17, 24, 50), (-24, -25, -27), (3, 21, 35), (-26, -36, 47), (-28, -45, 49), (-6, 12, -21), (-15, -17, -39), (2, -14, 41), (-23, 25, 36), (-3, -39, -40), (20, 35, 50), (27, 31, -39), (-15, -40, 45), (34, 35, 50), (-1, 12, -48), (18, -30, -35), (-24, -25, 27), (-4, -12, -33), (-24, -37, -43), (31, -37, -44), (-9, 14, -38), (-16, 33, 34), (4, -5, -35), (-3, -19, -21), (-29, -35, -36), (7, 36, -43), (14, 30, 41), (-7, -24, -35), (6, 35, -42), (-1, -15, 39), (-16, 27, 49), (-10, -37, 49), (-3, -